# Fixed Point Mapping for a Driven Nonlinear Oscillator

This notebook generates a map of fixed points for a driven nonlinear oscillator across parameter space. The system exhibits bistability, meaning it has multiple stable states (fixed points) for certain parameter ranges.

## System Parameters
- `epsilon`: Drive strength
- `kappa`: Damping rate
- `delta`: Detuning from resonance
- `chi`: Nonlinearity coefficient

In [ ]:
from __future__ import annotations
import numpy as np

from metastable.zero_damping import solve_zero_damping
from metastable.map.map import PhaseSpaceMap
from metastable.extend_map import extend_map

## Initialize Parameter Space

We create a grid in parameter space (epsilon, kappa) where we'll search for fixed points.
The system exhibits three types of fixed points:
1. Bright state (high amplitude)
2. Dim state (low amplitude)
3. Saddle point (unstable)

In [ ]:
seed_map = PhaseSpaceMap(
    epsilon_linspace=np.linspace(start=0.0, stop=30.0, num=601),  # Drive strength
    kappa_linspace=np.linspace(start=0.0, stop=5.0, num=401),     # Damping rate
    delta=7.8,    # Detuning
    chi=-0.1,     # Nonlinearity
)

## Generate Seed Solution

We start by finding fixed points at zero damping (kappa=0) where analytical solutions exist.
These solutions will serve as seeds for numerical continuation to non-zero damping.

In [ ]:
# Choose a point in parameter space for the seed solution
epsilon_idx = 0
kappa_idx = 0

# Double check that we are at zero damping
assert seed_map.kappa_linspace[kappa_idx] == 0.0

## Find Analytical Solutions

At zero damping, we can solve for fixed points analytically using the `solve_zero_damping` function.

In [ ]:
# Generate the seed solution analytically
seed_points = solve_zero_damping(
    epsilon=seed_map.epsilon_linspace[epsilon_idx],
    delta=seed_map.delta,
    chi=seed_map.chi,
)

# We need to start with seeds for all three types of fixed point
assert len([point for point in seed_points if point is not None]) == 3

## Update Map with Seeds

Store the analytical solutions in our map data structure.

In [ ]:
# Update the state of the arrays
seed_map.update_map(
    epsilon_idx=epsilon_idx, kappa_idx=kappa_idx, new_fixed_points=seed_points
)

## Extend Solutions Across Parameter Space

Using numerical continuation, we extend our solutions from zero damping to the full parameter space.
The `extend_map` function follows the fixed points as parameters change.

In [ ]:
fixed_points_map = extend_map(seed_map)

fixed_points_map.save_state(file_path="map-601x401.npz")